# **Jowar plant leaf disease detection**

## **Overview<font color='#FFCC00'>  • </font>**

The main aim of these project is to re-develope a new model for Jowar plant leaf disease detection, which is previosuly developed @Manivarsh-adi [github_page](https://github.com/Manivarsh-adi/Jowar_Plant_Leaf_Disease_Detection_Using_Deep_Learning)

After going through [Paper](https://ieeexplore.ieee.org/abstract/document/9596535) and above github page it's evident developed model is failing on production data (test data). So our motive is to develope a new model to boost up the testing score.

## **Previous work overview<font color='#FFCC00'>  • </font>**

[Paper](https://ieeexplore.ieee.org/abstract/document/9596535) defined a ResNet50 and self designed models to detect a jowar leaf is diagnosed with Athracnose and Leaf Blight or Healthy Leaf.

Author used a self designed image segemention technique to highlight symptoms of Anthracnose and leaf blight using OpenCV, this technique is inspired from google maps segementation technique.

![Segemented image](https://github.com/Manivarsh-adi/Jowar_Plant_Leaf_Disease_Detection_Using_Deep_Learning/blob/main/All_Segmentation.jpg?raw=true)



## **Project Motive**

Our motive is to build a new models cosidering the base models like LeNet-5 and ResNet50 for binary detection on the dataset. As seen model is failing to achieve good accuracy due to multi class classification. So we decided to consider a binary classification model where data is labeled as 0 (Healthy) and 1 (Diagnosed).

 $0-->Healthy$ and  $1-->Diagnosed$
 
 Instead of Symptoms threshold technique used by Author, we decide to go with grey scale images. As seen above and dataset for some of the images background is also marking as symptoms like soil colour etc..

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import datetime
import os


import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import models
from tensorflow.keras.layers import Flatten, BatchNormalization, Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
import keras
#from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l1, l2, l1_l2
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import experimental, layers
from tensorflow import keras

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

## Reading Data

In [3]:
train_dataset = image_dataset_from_directory(r"C:\Users\adhim\Desktop\Jowar Compressed Segemented\CompressedSegmentedGray\Train",
                                          image_size = (224, 224),
                                          #batch_size = 32,
                                          shuffle = True
                                          )

Found 323 files belonging to 2 classes.


In [4]:
val_dataset = image_dataset_from_directory( r"C:\Users\adhim\Desktop\Jowar Compressed Segemented\CompressedSegmentedGray\Validation",
                                                image_size = (224, 224),
                                                #batch_size = 32,
                                                shuffle = True
                                          )

Found 87 files belonging to 2 classes.


In [36]:
test_dataset = image_dataset_from_directory( r"C:\Users\adhim\Desktop\test images",
                                                image_size = (224, 224),
                                                #batch_size = 32,
                                                shuffle = True
                                          )

Found 96 files belonging to 2 classes.


## Splitting Data

In [6]:
x_train = []
y_train = []

for image_batch, label_batch in train_dataset:
    for image, label in zip(image_batch, label_batch):
        x_train.append(np.array(image.numpy()).astype(np.float32))
        y_train.append(np.array(label.numpy()).astype(np.int32))

In [7]:
x_val = []
y_val = []

for image_batch, label_batch in val_dataset:
     for image, label in zip(image_batch, label_batch):
            x_val.append(np.array(image.numpy()).astype(np.float32))
            y_val.append(np.array(label.numpy()).astype(np.int32))

In [16]:
x_test = []
y_test = []

for image_batch, label_batch in test_dataset:
     for image, label in zip(image_batch, label_batch):
            x_test.append(np.array(image.numpy()).astype(np.float32))
            y_test.append(np.array(label.numpy()).astype(np.int32))

## Replicating LeNet-5

Here our work is to replicate LeNet-5 to pre-train the dataset on this model.

### Lets look layer to layer feature map process

C1 layer-convolutional layer:
* Input picture: 250 * 250
* Convolution kernel size: 5 * 5
* Convolution kernel types: 6
* Output featuremap size: 246 * 246 (250 - 5 + 1) = 246
* Number of neurons: 246 246 6

S2 layer-pooling layer (downsampling layer):
* Input: 246 * 246
* Sampling area: 2 * 2
* Sampling method: 4 inputs are added, multiplied by a trainable parameter, plus a trainable offset. Results via sigmoid
* Sampling type: 6
* Output featureMap size: 123 * 123 (123/2)
* Number of neurons: 123 123 6

C3 layer-convolutional layer:
* Input: all 6 or several feature map combinations in S2
* Convolution kernel size: 5 * 5
* Convolution kernel type: 16
* Output featureMap size: 10 * 10 (123 - 5 + 1) = 118
* Number of neurons: 118 118 16

S4 layer-pooling layer (downsampling layer)
* Input: 118 * 118
* Sampling area: 2 * 2
* Sampling method: 4 inputs are added, multiplied by a trainable parameter, plus a trainable offset. Results via sigmoid
* Sampling type: 16
* Output featureMap size: 59 * 59 (118/2)
* Number of neurons: 59 59 16

C5 layer-convolution layer
* Input: All 16 unit feature maps of the S4 layer (all connected to s4)
* Convolution kernel size: 5 * 5
* Convolution kernel type: 120
* Output featureMap size: 55 * 55 (59-5 + 1)
* Number of neurons: 55 55 120

In [9]:
leaky_relu_alpha = 0.1

In [10]:
model = models.Sequential()

In [11]:
model.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'))

model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.2))

model.add(layers.Conv2D(filters = 6, kernel_size = (5, 5), strides = (1,1), padding = 'valid', activation = keras.activations.tanh, input_shape = (250, 250, 1)))

model.add(layers.MaxPool2D( pool_size = (2, 2), strides = (2, 2)))

model.add(layers.Conv2D(filters = 16, kernel_size = (5, 5), strides = (1, 1), padding = 'valid', activation = keras.activations.tanh))

model.add(layers.MaxPool2D( pool_size = (2, 2), strides = (2, 2)))

model.add(layers.Conv2D(filters = 120, kernel_size = (5, 5), strides = (1,1), padding = 'valid', activation = keras.activations.tanh))

model.add(layers.Flatten())

model.add(layers.Dense(84, activation = keras.activations.tanh))

model.add(layers.Dense(1, activation = keras.activations.softmax))

In [12]:
model.compile(loss=keras.metrics.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [13]:
model.fit(train_dataset, batch_size=128, epochs=20, verbose=1, validation_data=val_dataset)

Epoch 1/20
11/11 [==============================] - 15s 281ms/step - loss: 2.8706 - accuracy: 0.7399 - val_loss: 3.6404 - val_accuracy: 0.6897
Epoch 2/20
11/11 [==============================] - 2s 156ms/step - loss: 2.9558 - accuracy: 0.7399 - val_loss: 3.3624 - val_accuracy: 0.6897
Epoch 3/20
11/11 [==============================] - 2s 168ms/step - loss: 2.7242 - accuracy: 0.7399 - val_loss: 3.0917 - val_accuracy: 0.6897
Epoch 4/20
11/11 [==============================] - 2s 157ms/step - loss: 2.5020 - accuracy: 0.7399 - val_loss: 2.8308 - val_accuracy: 0.6897
Epoch 5/20
11/11 [==============================] - 2s 163ms/step - loss: 2.2791 - accuracy: 0.7399 - val_loss: 2.5537 - val_accuracy: 0.6897
Epoch 6/20
11/11 [==============================] - 2s 155ms/step - loss: 2.0390 - accuracy: 0.7399 - val_loss: 2.2703 - val_accuracy: 0.6897
Epoch 7/20
11/11 [==============================] - 2s 170ms/step - loss: 1.8093 - accuracy: 0.7399 - val_loss: 1.9920 - val_accuracy: 0.6897
Epoch

In [37]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)

print('\nTest accuracy:', test_acc)

3/3 - 0s - loss: 0.6893 - accuracy: 0.6250

Test accuracy: 0.625


## Fine tuning ResNet50 

Here our aim id to fine tune ResNet50 with binary grey scale image data using transer learning mechanism.

1. Firslty, need to load renet model from keras module by mentioning input size and as well as which weights need to consider to fine tune in this case we considered ever green "imagenet" weights. <br>
2. Using transfer learning mechanism we can allow some of the top layers to update during fin tuning process, here we allowed top two layers 144 and 145 layers to update during fine tuning.<br>
3. Then we will load sequential model by adding all necassery layers to it.
4. To make dataset large we try to make some flips on data which creates a new images for each image.
5. Then loaded resnet model is added to sequential layer.
6. Following added flatten layer to flatten the input to feed forward network.

## ResNet-50

In [22]:
input_x = tf.keras.Input( shape = (224, 224, 3))

resnet_model = tf.keras.applications.ResNet50(include_top = False,
                                               weights = 'imagenet',
                                               input_tensor = input_x)

In [23]:
for layers in resnet_model.layers[:143]:
    
    layers.trainable = False

In [24]:
model2 = models.Sequential()

model2.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'))

model2.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.2))

model2.add(resnet_model)

model2.add(Flatten())

#model2.add(Dense(84, activation = keras.activations.tanh))

#model2.add(Dense(1, activation = keras.activations.softmax))

model2.add(BatchNormalization())

model2.add(Dense(256, activation = 'relu'))

model2.add(Dropout(0.5))

model2.add(BatchNormalization())

model2.add(Dense(128, activation = 'relu'))

model2.add(Dropout(0.5))

model2.add(BatchNormalization())

model2.add(Dense(64, activation = 'relu'))

model2.add(Dropout(0.5))

model2.add(BatchNormalization())

model2.add(Dense(1, activation = 'softmax'))

In [25]:
model2.compile(loss = keras.metrics.binary_crossentropy,
              optimizer = 'adam',
              metrics = ['accuracy'])

In [26]:
history = model2.fit(train_dataset,
                    batch_size = 32,
                    epochs = 20,
                    validation_data = val_dataset)

Epoch 1/20
11/11 [==============================] - 21s 1s/step - loss: 0.8403 - accuracy: 0.7399 - val_loss: 1.1354 - val_accuracy: 0.6897
Epoch 2/20
11/11 [==============================] - 9s 789ms/step - loss: 0.6968 - accuracy: 0.7399 - val_loss: 3.3837 - val_accuracy: 0.6897
Epoch 3/20
11/11 [==============================] - 9s 778ms/step - loss: 0.6636 - accuracy: 0.7399 - val_loss: 9.9357 - val_accuracy: 0.6897
Epoch 4/20
11/11 [==============================] - 9s 780ms/step - loss: 0.5790 - accuracy: 0.7399 - val_loss: 12.0524 - val_accuracy: 0.6897
Epoch 5/20
11/11 [==============================] - 9s 778ms/step - loss: 0.4917 - accuracy: 0.7399 - val_loss: 7.4145 - val_accuracy: 0.6897
Epoch 6/20
11/11 [==============================] - 9s 777ms/step - loss: 0.5315 - accuracy: 0.7399 - val_loss: 9.6291 - val_accuracy: 0.6897
Epoch 7/20
11/11 [==============================] - 9s 776ms/step - loss: 0.5076 - accuracy: 0.7399 - val_loss: 6.3884 - val_accuracy: 0.6897
Epoch 8

In [38]:
test_loss, test_acc = model2.evaluate(test_dataset, verbose=2)

print('\nTest accuracy:', test_acc)

3/3 - 0s - loss: 0.5662 - accuracy: 0.6250

Test accuracy: 0.625
